# snowmobile

```{include} /description.md
```

```{eval-rst}

.. toctree::
    :maxdepth: 1
    :hidden:

    ./setup.md

.. toctree::
    :caption: Usage
    :maxdepth: 1
    :hidden:

    ./usage/snowmobile.md
    ./usage/table.md
    ./usage/script.md
    ./usage/sql.md
    ./usage/snowmobile_toml.md

.. toctree::
    :caption: Additional Resources
    :maxdepth: 1
    :hidden:

    ./modindex.md
    ./extensions.md
    ./usage/advanced.md
    ./snippets.md

.. toctree::
    :caption: Meta
    :maxdepth: 1
    :hidden:

    ./acknowledgements.md
    ./changelog.md
    ./authors.md
    ./license.md
```

## Example
- *[Connecting](#connecting)*
- *[Loading Data](#loading-data)*
- *[Executing Queries](#executing-queries)*
- *[Working with SQL Scripts](#working-with-sql-scripts)*
- *[Information API](#information-api)*

<br>

### *Connecting*
````{admonition} snowmobile.connect()
:class: toggle, note, is_explanation, sn-dedent-v-container

```{div} sn-dedent-v
<hr class="sn-green">
```

```{div} sn-dedent-v-t, sn-dedent-v-b-h
{func}`snowmobile.connect()` is an alias for {class}`snowmobile.Snowmobile()`; its purpose is to:
```
1.  Locate, instantiate, and store [snowmobile.toml](./usage/snowmobile_toml.md#snowmobiletoml)
    as a {class}`~snowmobile.Configuration` object ({class}`sn.cfg`)
1.  Establish a connection to {xref}`snowflake` and store the {xref}`SnowflakeConnection` ({class}`sn.con`)
1.  Serve as the primary entry point to the {xref}`SnowflakeConnection` and {xref}`snowmobile` APIs
+++
```{div} sn-dedent-v-t-h
  The first time it's invoked, [Snowmobile](./usage/snowmobile.md) will find [snowmobile.toml](./usage/snowmobile_toml) and cache its location;
  this step isn't repeated unless the file is moved, the cache is manually cleared, or a new version of {xref}`snowmobile` is installed.
```
+++
**With all arguments omitted, it will authenticate with the default credentials and connection arguments specified in** [**snowmobile.toml**](./usage/snowmobile_toml).

<hr class="sn-green">

{link-badge}`./usage/snowmobile.html,cls=badge-primary badge text-white,Usage: snowmobile.Snowmobile,tooltip=Intro & usage documentation for snowmobile.Snowmobile`
{link-badge}`./autoapi/snowmobile/core/connection/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.connection,tooltip=Documentation parsed from docstrings`
````

```{div} sn-pre-code, sn-dedent-v-t-h
Establishing a connection based on configured defaults is done with:
```

In [16]:
import snowmobile

sn = snowmobile.connect()

Locating credentials..
(1 of 2) Finding snowmobile.toml..
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='creds1')


```{div} sn-pre-code
Which gives us:
```

In [6]:
print(sn)            #> snowmobile.Snowmobile(creds='creds1')
print(sn.cfg)        #> snowmobile.Configuration('snowmobile.toml')
print(type(sn.con))  #> <class 'snowflake.connector.connection.SnowflakeConnection'>

snowmobile.Snowmobile(creds='creds1')
snowmobile.Configuration('snowmobile.toml')
<class 'snowflake.connector.connection.SnowflakeConnection'>


```{div} sn-pre-code
We can verify the connection with:
```

In [ ]:
print(sn.alive)  #> True

<br>

### *Loading Data*

`````{admonition} snowmobile.Table
:class: toggle, is_explanation, sn-dedent-v-t-container, sn-dedent-v-b-h-container

````{tabbed} Context

  {class}`~snowmobile.Table` is a loading solution that at minimum accepts a
  [**Snowmobile**](./usage/snowmobile) ({class}`sn`), a {class}`~pandas.DataFrame` ({class}`df`),
  and a table name ({class}`str`).
+++
  In the same way that [**Snowmobile**](./usage/snowmobile) handles its keyword arguments,
  {class}`~snowmobile.Table` will adhere to any arguments explicitly provided and defer
  to the values configured in [snowmobile.toml](./usage/snowmobile_toml) otherwise.
+++
  `as_is=True` tells `t1` to invoke {meth}`~snowmobile.Table.load()` with
  the arguments provided as opposed to inspecting `t1` for information on the existence
  of the table or its compatability with the {class}`df` we're loading.

````

````{tabbed} +
  ```{div} sn-dedent-v-b-h, sn-dedent-v-t
  *The behavior outlined below reflects those within the
  [default snowmobile.toml file](./usage/snowmobile_toml.md#file-contents)*, meaning that `t1` will:
  ```

  1. Check if *sample_table* exists in the schema associated with {attr}`sn.con`
  2. If *sample_table* **does** exist, it will validate `df` against *sample_table* and throw an error
     if their dimensions are not identical
  3. If *sample_table* does **not** exist (as is the case here), it will generate DDL from `df` and execute it as part of the loading process
````

<hr class="sn-green">

{link-badge}`./usage/table.html,cls=badge-primary badge text-white,Usage: snowmobile.Table,tooltip=Intro & usage documentation for snowmobile.Table`
{link-badge}`./autoapi/snowmobile/core/table/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.table,tooltip=Documentation parsed from docstrings`
`````

````{admonition} Setup
:class: toggle, is-setup, sn-dedent-v-t-h-container, sn-clear-title

```{div} sn-dedent-v-t-h, hanging
The `df` used below can be created with:
```

```python
import pandas as pd
import numpy as np

df = pd.DataFrame(
  data = {'col1': np.ones(3), 'col2': np.zeros(3)}
)

print(df.shape)  #> (3, 2)
```

<hr class="sn-spacer-thin">

````

<br>

### *Loading Data*

`````{admonition} snowmobile.Table
:class: toggle, is_explanation, sn-dedent-v-t-container, sn-dedent-v-b-h-container

````{tabbed} Context
  {class}`~snowmobile.Table` is a loading solution that at minimum accepts a
  [**Snowmobile**](./usage/snowmobile) ({class}`sn`), a {class}`~pandas.DataFrame` ({class}`df`),
  and a table name ({class}`str`).
+++
  In the same way that [**Snowmobile**](./usage/snowmobile) handles its keyword arguments,
  {class}`~snowmobile.Table` will adhere to any arguments explicitly provided and defer
  to the values configured in [snowmobile.toml](./usage/snowmobile_toml) otherwise.
````

````{tabbed} +
  ```{div} sn-dedent-v-b-h, sn-dedent-v-t
  *The behavior outlined below reflects those within the
  [default snowmobile.toml file](./usage/snowmobile_toml.md#file-contents)*, meaning that `t1` will:
  ```

  1. Check if *sample_table* exists in the schema associated with {attr}`sn.con`
  2. If *sample_table* **does** exist, it will validate `df` against *sample_table* and throw an error
     if their dimensions are not identical
  3. If *sample_table* does **not** exist (as is the case here), it will generate DDL from `df` and execute it as part of the loading process
````

<hr class="sn-green">

{link-badge}`./usage/table.html,cls=badge-primary badge text-white,Usage: snowmobile.Table,tooltip=Intro & usage documentation for snowmobile.Table`
{link-badge}`./autoapi/snowmobile/core/table/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.table,tooltip=Documentation parsed from docstrings`
`````

````{admonition} Setup
:class: toggle, is-setup, sn-dedent-v-t-h-container, sn-clear-title

```{div} sn-dedent-v-t-h, hanging
The `df` used below can be created with:
```

```python
import pandas as pd
import numpy as np

df = pd.DataFrame(
  data = {'COL1': np.ones(3), 'COL2': np.zeros(3)}
)

print(df.shape)  #> (3, 2)
```

<hr class="sn-spacer-thin">

````

In [12]:
# Verify table does not exist before moving forward with example
if sn.sql.exists('sample_table'):
    sn.sql.drop('sample_table')

In [26]:
"""Generate dummy DataFrame for snowmobile.Table example."""
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {'COL1': np.ones(3), 'COL2': np.zeros(3)}
)
df.head()

,COL1,COL2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


```{div} sn-indent-h-cell
<hr class="sn-green" style="margin-top: 0.5rem; margin-bottom: -0.2rem;">
```

```{div} sn-pre-code, sn-indent-v-t-h-container
Given a `sn` ({class}`~snowmobile.Snowmobile`), a `df` ({class}`~pandas.DataFrame`), and a `table` ({class}`str`) to load into, a {class}`~snowmobile.Table` can be created with:
```

<hr class="sn-spacer">

```{div} sn-pre-code, sn-indent-v-t-h-container
Given a `sn` ({class}`~snowmobile.Snowmobile`), a `df` ({class}`~pandas.DataFrame`), and a `table` ({class}`str`) to load into, a {class}`~snowmobile.Table` can be created with:
```

In [17]:
t1 = snowmobile.Table(sn=sn, df=df, table='sample_table')

```{div} sn-pre-code
With the default configuration, `t1` has gone ahead and pre-inspected some things like:
```

In [18]:
print(t1.exists)  #> False  -> 'sample_table' doesn't exist in the schema associated with 'sn'

False


```{div} sn-pre-code
We can create *sample_table* and load `df` into with:
```

In [19]:
t1.load()

```{div} sn-pre-code
And do some inspection on *sample_table* that was just created with:
```

In [19]:
print(t1.loaded)                           #> True -> t1 didn't encounter any errors during .load()
print(sn.sql.cnt_records('sample_table'))  #> 3  ---> 'sample_table' contains three records

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds


snowmobile.Table(table='SAMPLE_TABLE')

```{div} sn-indent-h-cell
<hr class="sn-green" style="margin-top: 1.4rem; margin-bottom: 0.5rem;">
```

```{div} sn-pre-code
The pre-inspection steps can be skipped and the same `df` appended to *sample_table* created by `t1.load()` with:
```

In [21]:
t2 = snowmobile.Table(sn=sn, df=df, table='sample_table', as_is=True)

```{div} sn-pre-code
Which can be verified in the same manner as was done after `t1.load()`:
```

In [21]:
print(t2.loaded)                           #> True
print(sn.sql.cnt_records('sample_table'))  #> 6

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 3)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(2 of 3)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(3 of 3)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds
True
6


```{div} sn-pre-code
Again with the default configuration and since *sample_table* was pre-existing when `t2` was created, it also made sure:
```

In [22]:
print(t2.cols_match)  #> True

True


```{div} sn-indent-h-cell
<hr class="sn-green" style="margin-top: 1.4rem; margin-bottom: 0.5rem;">
```

```{div} sn-pre-code
Let's create one more {class}`~pandas.DataFrame` but with different dimensions than `df`:
```

In [27]:
df2 = pd.concat([df, df], axis=1)
df2.head(2)

,COL1,COL2,COL1,COL2
0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0


```{div} sn-pre-code
And with it create another {class}`~snowmobile.Table`, `t3`, in the same way we did `t1`:
```

In [24]:
t3 = snowmobile.Table(sn=sn, df=df2, table='sample_table')

```{div} sn-pre-code
Here's what `t3` knows about `df` and *sample_table*:
```

In [25]:
print(t3.exists)      #> True  --> 'sample_table' exists in the schema associated with `sn`
print(t3.cols_match)  #> False  -> `df2` and 'sample_table' don't share the same dimensions

```{div} sn-pre-code
{class}`~snowmobile.Table` also checked the columns of `df2` and deduped them by appending a suffix to the duplicate set:
```

In [28]:
t3.df.head(2)

,COL1,COL2,COL1_1,COL2_1
0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0


```{div} sn-pre-code
If we try to load it the same way we did the others, we'll get an error with the default [snowmobile.toml](./usage/snowmobile_toml):
```

In [33]:
from snowmobile.core.errors import ColumnMismatchError

try:
    t3.load()
except ExistingTableError as e:
    print(e)
"""
ColumnMismatchError: `SAMPLE_TABLE` columns do not equal those in the local DataFrame 
and if_exists='append' was specified.
Either provide if_exists='replace' to overwrite the existing table or see `table.col_diff`
to inspect the mismatched columns.
"""

ColumnMismatchError: `SAMPLE_TABLE` columns do not equal those in the local DataFrame and if_exists='append' was specified.
Either provide if_exists='replace' to overwrite the existing table or see `table.col_diff` to inspect the mismatched columns.

```{div} sn-pre-code
Explicit arguments always take precedent over [snowmobile.toml](./usage/snowmobile_toml), so *sample_table* can be recreated and `df2` loaded into it with:
```

In [32]:
t3.load(if_exists='replace')
print(t3.loaded)  #> True

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 3 seconds
True


<br>

### *Executing Queries*

`````{admonition} sn: snowmobile.Snowmobile
:class: toggle, is_explanation, sn-dedent-v-t-container, sn-dedent-v-b-h-container

```{div} sn-dedent-t-h, hanging
  {xref}`snowmobile` provides three convenience methods for executing raw SQL directly off `sn`:
```

{meth}`~snowmobile.Snowmobile.query()` implements {meth}`pandas.read_sql()` for querying results into a {class}`~pandas.DataFrame`\
{link-badge}`./autoapi/snowmobile/core/connection/index.html#snowmobile.core.connector.Snowmobile.query,cls=badge-secondary badge-pill text-white,API Docs: Snowmobile.query(),tooltip=Documentation parsed from module docstring`

+++

{meth}`~snowmobile.Snowmobile.ex()` implements {meth}`SnowflakeConnection.cursor().execute()` for executing commands within a {xref}`SnowflakeCursor`\
{link-badge}`./autoapi/snowmobile/core/connection/index.html#snowmobile.core.connector.Snowmobile.ex,cls=badge-secondary badge-pill text-white,API Docs: Snowmobile.ex()   ,tooltip=Documentation parsed from module docstring`

+++

{meth}`~snowmobile.Snowmobile.exd()` implements {meth}`SnowflakeConnection.cursor(DictCursor).execute()` for executing commands within a {xref}`DictCursor`\
{link-badge}`./autoapi/snowmobile/core/connection/index.html#snowmobile.core.connector.Snowmobile.exd,cls=badge-secondary badge-pill text-white,API Docs: Snowmobile.exd(),tooltip=Documentation parsed from module docstring`

+++

<hr class="sn-green">

{link-badge}`./usage/snowmobile.html#executing-raw-sql,cls=badge-primary badge text-white,Usage: Executing Raw SQL,tooltip=Usage documentation for Executing Raw SQL`
{link-badge}`./autoapi/snowmobile/core/connection/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.connection,tooltip=Documentation parsed from docstrings`

`````

In [5]:
# Into a DataFrame
sn.query('select * from sample_table')

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


In [16]:
# Into a SnowflakeCursor
sn.ex('select * from sample_table').fetchall()

[(1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]

In [20]:
# Into a DictCursor
sn.exd('select * from sample_table').fetchall()

[{'COL1': 1.0, 'COL2': 0.0},
 {'COL1': 1.0, 'COL2': 0.0},
 {'COL1': 1.0, 'COL2': 0.0}]

### *Working with SQL Scripts*

`````{admonition} snowmobile.Script
:class: toggle, is_explanation

```{div} sn-dedent-v-t-h, sn-dedent-v-b-h
[**snowmobile.Script**](./usage/script.ipynb) imports a sql file and parses its contents according to its structure.
```

At a minimum, the file is split into individual statements, each of which is checked for tags;
if none are found, [Script](./usage/script.ipynb) will generate a generic name for the statement based on
the literal first SQL keyword it contains and its index position (e.g. `select data~statement #3` below).

+++

```{div} sn-dedent-v-b-h
By providing {class}`script` (below) the same instance of {class}`sn` with which {class}`t1` (above)
was instantiated, **the {xref}`SnowflakeConnection` and [Configuration](./usage/snowmobile_toml) is
shared amongst:**
```

- {class}`sn:` {class}`~snowmobile.Snowmobile`
- {class}`t1:` {class}`~snowmobile.Table`
- {class}`script:` {class}`~snowmobile.Script`


<hr class="sn-green">

{link-badge}`./usage/script.html,cls=badge-primary badge text-white,Usage: snowmobile.Script,tooltip=Intro & usage documentation for snowmobile.Script`
{link-badge}`./autoapi/snowmobile/core/script/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.script,tooltip=Documentation parsed from docstrings`

`````

````{admonition} Setup
:class: toggle, is-setup, toggle-shown, sn-dedent-v-t-h-container, sn-clear-title

```{div} sn-dedent-v-t-h, hanging
The `path` used below is a full path ({class}`str` or {class}`~pathlib.Path`) to **sample_table.sql**, the contents of which are:
```

```{literalinclude}  ./snippets/getting_started/sample_table.sql
:language: sql
:lines: 1-17
```

<hr class="sn-spacer-thin">

````

In [6]:
# Setup
from pathlib import Path

path = Path.cwd() / 'snippets' / 'snowmobile' / 'sample_table.sql'

<p style="margin-left: 0.9rem; margin-top: 1em; font-size: 0.75rem; font-weight: 800;">Setup</p>

```{div} sn-indent-cell, sn-pre-cell
Setup
```

In [12]:
script = snowmobile.Script(sn=sn, path=path)
print(script)

snowmobile.Script('sample_table.sql')


In [13]:
# High-level view of parsed contents
script.dtl()

sample_table.sql
1: Statement('create transient table~any_random_table1')
2: Statement('create table~sample_table')
3: Statement('select data~statement #3')
4: Statement('select data~sample_table')


<p class="sn-indent-cell">Accessing Statements</p>

In [14]:
# Individual statements can be accessed by index position
script(4)

Statement('select data~sample_table')

In [15]:
# ..including negative indexing
script(-1)

Statement('select data~sample_table')

In [16]:
# ..or by their tag as a string
script('select data~sample_table')

Statement('select data~sample_table')

<p class="sn-indent-cell">Executing Statements</p>

In [41]:
# They can be run directly off the Script
script.run(4, render=True)
script(4).results.head(1)

```sql
select * from sample_table;
```

,col1,col2
0,1.0,0.0


In [27]:
# ..or stored and manipulated on their own
s4 = script(4).run(render=True)
s4.results.head(1)

```sql
select * from sample_table;
```

,col1,col2
0,1.0,0.0


&nbsp;&nbsp;See [Advanced Examples](./usage/advanced.md) for more in-depth applications of {class}`~snowmobile.Script`.

In [40]:
with script.filter(incl_kw=['create'], excl_nm=['create table']) as s:
    s.dtl()

print('')

script.dtl()

sample_table.sql
1: Statement('create transient table~any_random_table1')

sample_table.sql
1: Statement('create transient table~any_random_table1')
2: Statement('create table~sample_table')
3: Statement('select data~statement #3')
4: Statement('select data~sample_table')


In [53]:
script(2).run(render=True)

```sql
create or replace table sample_table (
	col1 int,
	col2 int
);
```

Statement('create table~sample_table')

In [34]:
print(script.s(2).sql)

create or replace table sample_table (
	col1 int,
	col2 int
)


In [35]:
print(script.s('create table~sample_table').sql)

create or replace table sample_table (
	col1 int,
	col2 int
)


In [23]:
with script.filter(incl_nm=['.*sample_table']) as s:
    s.dtl()

sample_table.sql
1: Statement('create table~sample_table')


<br>

### *Information API*

`````{admonition} snowmobile.SQL
:class: toggle, is_explanation

```{div} sn-dedent-v-t
{class}`snowmobile.SQL` generates and executes raw SQL from inputs;
it comes free as the {attr}`~snowmobile.Snowmobile.sql` attribute of
{class}`~snowmobile.Snowmobile`, and its purpose is to provide a simple
Python API to query metadata and execute administrative commands
against {xref}`snowflake`.
```

<hr class="sn-blue">

{link-badge}`./usage/sql.html,cls=badge-primary badge text-white,Usage Docs: snowmobile.SQL,tooltip=Intro & usage documentation for snowmobile.SQL`
{link-badge}`./autoapi/snowmobile/core/sql/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.sql,tooltip=Documentation parsed from docstrings`

`````

<p class="sn-indent-cell"></p>

In [5]:
# Check existence of tables/views
sn.sql.exists('sample_table')

True

In [2]:
# Sample 'n' records from a table
sn.sql.table_sample('sample_table', n=-1)

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


In [3]:
# Submit basic administrative commands
sn.sql.clone(nm='sample_table', to='sample_table2')

,status
0,Table SAMPLE_TABLE2 successfully created.


In [4]:
# Check depth
sn.sql.cnt_records('sample_table2')

3

In [5]:
# Fetch columns
sn.sql.columns('sample_table2')

['COL1', 'COL2']

In [53]:
# Query DDL
print(sn.sql.ddl('sample_table'))

create or replace TABLE SAMPLE_TABLE (
	COL1 FLOAT,
	COL2 FLOAT
);


In [7]:
# Provide `run=False` to get the raw sql
drop_sql = sn.sql.drop('sample_table', run=False)

print(type(drop_sql))
print(drop_sql)

<class 'str'>
drop table if exists GEM7318.SAMPLE_TABLE


<p class="sn-indent-cell"></p>

<p class="sn-indent-cell">Cleaning up after ourselves can be done with:</p>

In [8]:
for t in ['sample_table', 'sample_table2']:
    sn.sql.drop(t)

<p class="sn-indent-cell"></p>

---

# <u>Meta / Non-Output</u>
---

All cells below this are either excluded from output via the `remove-cell` cell-tag
or contain contents that will not visibly render in the output.

# Style
---

Reduce vertical spacing between h2-h3 headings for *Examples*:
```css
<style>
.md-typeset h2, .md-typeset h3 {
    margin-top: -0.5rem;
}
</style>
```

<style>
.md-typeset h1, .md-typeset h2 {
    margin-top: -0.5rem;
}
    
.md-typeset h3 {
    margin-top: -0.5rem;
    margin-bottom: 0.2rem;
    font-size: 140%;
}
    
.md-typeset .admonition.is-setup {
    border-left-color: #11838e;
    margin: -0.15rem 1.5rem .7rem 0.85rem;
    display: block;
}
</style>